In [ ]:
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import PythonScriptStep

ws = Workspace.get(name="quick-starts-ws-126720")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
vm_size = "Standard_D2_V2"
compute_name="FirstComp"
# create the cluster
compute_min_nodes=0
compute_max_nodes=4
 



### YOUR CODE HERE ###
# Checking if  compute already exist or not  if not creating a new target
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())



In [4]:
print(compute_name)

FirstComp


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( { 
    "--C": choice([0.00001,0.001,0.1,1,10,50,100,10000]),
     "--max_iter": choice(range(10, 200))     } )


# Specify a Policy

policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
'''
if "training" not in os.listdir():
    os.mkdir("./training")
'''

'\nif "training" not in os.listdir():\n    os.mkdir("./training")\n'

In [6]:
# Create a SKLearn estimator for use with train.py
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
import shutil
#shutil.copy('./train.py',source_directory)

est = SKLearn(source_directory="./", 
                compute_target=compute_target, 
                vm_size=vm_size,
                entry_script='train.py'
               )


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=8,
                                max_concurrent_runs=4)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

exp_hyper=Experiment(ws,"hyper_test")
model=exp_hyper.submit(hyperdrive_config,show_output=True)
model.wait_for_completion(show_output=True)
best_run=model.get_best_run_by_primary_metric()
best_run.get_metrics()





WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


RunId: HD_71320690-052e-438b-846b-1b9e6b581804
Web View: https://ml.azure.com/experiments/hyper_test/runs/HD_71320690-052e-438b-846b-1b9e6b581804?wsid=/subscriptions/de47103e-2da6-4f5e-88fc-d18b27fd249b/resourcegroups/aml-quickstarts-126720/workspaces/quick-starts-ws-126720

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-18T03:03:50.302775][API][INFO]Experiment created<END>\n""<START>[2020-11-18T03:03:51.033396][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-11-18T03:03:51.389322][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-11-18T03:03:52.7988871Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_71320690-052e-438b-846b-1b9e6b581804
Web View: https://ml.azure.com/experiments/hyper_test/runs/HD_71320690-052e-438b-846b-1b9e6b581804?wsid=/subscriptions/de47103e-2da6-4

{'Regularization Strength:': 0.001,
 'Max iterations:': 34,
 'Accuracy': 0.9044006069802731}

In [8]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyper_test,HD_71320690-052e-438b-846b-1b9e6b581804_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
#Used RunDetails to get which run id and its parameter values
from azureml.widgets import RunDetails
RunDetails(model).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
best_run=model.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run.get_metrics()['Accuracy'])
print('\n Accuracy:', best_run.get_metrics()['Regularization Strength:'])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

Best Run Id:  HD_71320690-052e-438b-846b-1b9e6b581804_1

 Accuracy: 0.9044006069802731

 Accuracy: 0.001


In [21]:
from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="log_model.pkl", model_name="test")

Registering model test


In [22]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
df=Dataset.Tabular.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [23]:
df.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [25]:
from azureml.train.automl import AutoMLConfig
import numpy as np


# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    compute_target=compute_target,
    primary_metric="accuracy",
    training_data=df,
    label_column_name="y",
    allowed_models=["LightGBM","RandomForest"],
    n_cross_validations=3)


In [26]:


# Submit your automl run

exp=Experiment(ws,"automl_test")
automl_model=exp.submit(automl_config,show_output=True)
### YOUR CODE HERE ###
import joblib
from azureml.widgets import RunDetails
RunDetails(automl_model).show()
best_run, fitted_model = automl_model.get_output()
print(best_run)
print(fitted_model)




Running on remote.
Running on remote compute: FirstComp
Parent Run ID: AutoML_32b6b3da-cbd9-4925-931c-98b06c840368

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bia

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run(Experiment: automl_test,
Id: AutoML_32b6b3da-cbd9-4925-931c-98b06c840368_27,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.29105263157894734,
                                                                                                    min_weight_fraction_leaf=0.0

In [27]:
#y_test = x_test.pop("totalAmount")
import joblib
from sklearn.model_selection import train_test_split
x_df = df.to_pandas_dataframe()
y=x_df.y
x=x_df.drop(['y'],axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1, random_state=42)
y_predict = fitted_model.predict(x_test)
#print(y_predict[:10])
from sklearn.metrics import accuracy_score
accuracy_score(y_predict,y_test)
joblib.dump(fitted_model,"auoml_best_model.pkl")

['auoml_best_model.pkl']

In [28]:
auto_ml = Model.register(workspace=ws, model_path='auoml_best_model.pkl', model_name="auto_ml")

Registering model auto_ml


In [29]:
compute_target.delete()